In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../data/train_modified.csv')
target = 'Disbursed'
IDcol = 'ID'
print("value counts:", train['Disbursed'].value_counts())

x_columns = [x for x in train.columns if x not in ['Disbursed', 'ID']]
x = train[x_columns]
y = train[target]

value counts: 0    19680
1      320
Name: Disbursed, dtype: int64


In [3]:
"""
分为测试数据和训练数据
"""
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(x_train, y_train)
y_pred = gbm0.predict(x_test)
y_predprob = gbm0.predict_proba(x_test)[:, 1]
print("Accuracy:%.4f" % metrics.accuracy_score(y_test, y_pred))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))

Accuracy:0.9865
AUC Score (Train): 0.877438


In [4]:
"""
网格搜索，调整learning_rate 和 迭代次数n_estimators
"""
param_test1 = {'n_estimators': range(20, 81, 10)}
gsearch1 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300, min_samples_leaf=20,
                                         max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),
    param_grid=param_test1, scoring='roc_auc', iid=False, cv=5)

gsearch1.fit(x_train, y_train)
print("grid_best_params:", gsearch1.best_params_)
print("grid_best_score:", gsearch1.best_score_)


grid_best_params: {'n_estimators': 40}
grid_best_score: 0.8187689965967673


In [ ]:
"""=====================================================================================================================
网格搜索，确定n_estimators为40 再做后面的选择;调整决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索
"""
param_test2 = {'max_depth': range(3, 14, 2), 'min_samples_split': range(100, 801, 200)}
gsearch2 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, min_samples_split=300, min_samples_leaf=20,
                                         max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),
    param_grid=param_test2, scoring='roc_auc', iid=False, cv=5)
gsearch2.fit(x_train, y_train)
print("grid_best_params:", gsearch2.best_params_)
print("grid_best_score:", gsearch2.best_score_)


In [5]:

"""
'max_depth':9; min_samples_split 暂时不能确定下来， 和min_samples_leaf 一起调参数
"""
param_test3 = {'min_samples_split': range(800, 1900, 200), 'min_samples_leaf': range(60, 101, 10)}
gsearch3 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, min_samples_split=300, min_samples_leaf=20,
                                         max_depth=9, max_features='sqrt', subsample=0.8, random_state=10),
    param_grid=param_test3, scoring='roc_auc', iid=False, cv=5)
gsearch3.fit(x_train, y_train)
print("grid_best_params:", gsearch3.best_params_)
print("grid_best_score:", gsearch3.best_score_)

grid_best_params: {'min_samples_leaf': 60, 'min_samples_split': 800}
grid_best_score: 0.8189044557938722


In [10]:
"""
 一.经过上面的网格搜索得到了参数：learning_rate=0.1;n_estimators=40;max_depth:9; min_samples_leaf:60 和 min_samples_split:800;
 查看最终的拟合效果
 """
gbm1 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, max_depth=7,min_samples_split=800, min_samples_leaf=60,
                                  max_features='sqrt',subsample=0.8,random_state=10)
gbm1.fit(x_train,y_train)
y_predict = gbm1.predict(x_test)
y_predprob = gbm1.predict_proba(x_test)[:,1]

print("Accuracy: %.6f" % metrics.accuracy_score(y_test, y_predict))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))

Accuracy: 0.987500
AUC Score (Train): 0.865276


In [7]:
"""
使用前面的参数，有一定的下降，这里只是用了sumsample=0.8, 接下来调整max_features
"""

param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, min_samples_split=300, min_samples_leaf=20,
                                         max_depth=7, max_features='sqrt', subsample=0.8, random_state=10),
    param_grid=param_test4, scoring='roc_auc', iid=False, cv=5)
gsearch4.fit(x_train, y_train)
print("grid_best_params:", gsearch4.best_params_)
print("grid_best_score:", gsearch4.best_score_)

grid_best_params: {'max_features': 15}
grid_best_score: 0.8218563175201872


In [8]:
"""
调整subsample
"""
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, min_samples_split=300, min_samples_leaf=20,
                                         max_depth=15, max_features='sqrt', subsample=0.8, random_state=10),
    param_grid=param_test5, scoring='roc_auc', iid=False, cv=5)
gsearch5.fit(x_train, y_train)
print("grid_best_params:", gsearch5.best_params_)
print("grid_best_score:", gsearch5.best_score_)

grid_best_params: {'subsample': 0.9}
grid_best_score: 0.8162177686088867


In [11]:
"""
二.再次查看调整参数后的结果
"""
gbm2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=40, max_depth=15,min_samples_split=800, min_samples_leaf=60,
                                  max_features='sqrt',subsample=0.9,random_state=10)
gbm2.fit(x_train,y_train)
y_predict = gbm2.predict(x_test)
y_predprob = gbm2.predict_proba(x_test)[:,1]

print("Accuracy: %.6f" % metrics.accuracy_score(y_test, y_predict))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))

Accuracy: 0.987500
AUC Score (Train): 0.856020


In [12]:
"""
AUC 相对之前的 有所下降， 这里主要是因为使用了80%的样本量
接下来调整参数 learning_rate 和 n_estimators, 分别减少一半
"""
gbm3 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=20, max_depth=15,min_samples_split=800, min_samples_leaf=60,
                                  max_features='sqrt',subsample=0.9,random_state=10)
gbm3.fit(x_train,y_train)
y_predict = gbm3.predict(x_test)
y_predprob = gbm3.predict_proba(x_test)[:,1]

print("Accuracy: %.6f" % metrics.accuracy_score(y_test, y_predict))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))


Accuracy: 0.987500
AUC Score (Train): 0.852071


In [15]:
"""
可以看到AUC分数比起之前的版本稍有下降，这个原因是我们为了增加模型泛化能力，为防止过拟合而减半步长，最大迭代次数加倍，同时减小了子采样的比例，从而减少了训练集的拟合程度。

　　　　下面我们继续将步长缩小5倍，最大迭代次数增加5倍，继续拟合我们的模型：
"""
gbm4 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, max_depth=15,min_samples_split=800, min_samples_leaf=60,
                                  max_features='sqrt',subsample=0.9,random_state=10)
gbm4.fit(x_train,y_train)
y_predict = gbm4.predict(x_test)
y_predprob = gbm4.predict_proba(x_test)[:,1]

print("Accuracy: %.6f" % metrics.accuracy_score(y_test, y_predict))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))


Accuracy: 0.987500
AUC Score (Train): 0.861347


In [18]:
"""
AUC 相对之前有所提升，最后我们继续步长缩小一半，获取最大的迭代次数，拟合我们的模型：
"""
param_test6 = {'n_estimators':range(300,1000,50)}
gsearch6 = GridSearchCV(
    estimator=GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, max_depth=15,min_samples_split=800, min_samples_leaf=60,
                                  max_features='sqrt',subsample=0.9,random_state=10),
    param_grid=param_test6, scoring='roc_auc', iid=False, cv=5)
gsearch6.fit(x_train, y_train)
print("grid_best_params:", gsearch6.best_params_)
print("grid_best_score:", gsearch6.best_score_)

grid_best_params: {'n_estimators': 600}
grid_best_score: 0.823275527836147
